In [11]:
import c_runner
c_runner.set_c_dir(r"F:\Documents\0827\Learning Something\MCU_Learning\蓝桥杯C51学习\Project\000LearningStart\03-Digital_Tube\User") 

# 数码管显示

## 静态显示讲解

### 最基础 --1位数码管静态显示

In [ ]:
%%c main
#include <STC15F2K60S2.H> // 包含寄存器定义
#include <intrins.h>      // 提供 _nop_() 延时

// 1. 定义锁存器“门牌号”
#define LATCH_DIG 0xC0  // 位选门：控制哪位亮
#define LATCH_SEG 0xE0  // 段选门：控制亮什么形状
#define LATCH_LED 0x80  // LED 门：控制小灯

typedef unsigned char u8;

/**
 * @brief 核心写总线函数
 * 就像是把信(dat)投递到特定的信箱(addr)里。
 */
void BusWrite(u8 addr, u8 dat)
{
    P0 = dat;        // 把数据放在 P0 总线上
    P2 = addr;       // 瞬间打开对应的锁存器闸门
    _nop_();         // 稍作停留，让硬件反应一下
    P2 = 0x00;       // 关闭闸门，数据就被“锁”在里面了
}

void main(void)
{
    // --- 第一步：初始化，把不需要的东西都关掉 ---
    BusWrite(LATCH_LED, 0xFF); // LED 全灭（1为灭）
    
    // --- 第二步：静态显示核心逻辑 ---
    
    // 1. 告诉硬件：我要选左起第 1 个数码管
    // 0x01 (0000 0001) 代表选中第 1 位
    BusWrite(LATCH_DIG, 0x01); 

    // 2. 告诉硬件：在这个位置显示数字 "8"
    // 0x80 是数字 "8" 的段码（除了 dp 位，其余 a-g 全亮）
    BusWrite(LATCH_SEG, 0x80); 

    // --- 第三步：原地踏步 ---
    while(1)
    {
        // 既然是静态显示，程序跑完上面两行就没事干了。
        // CPU 会一直停在这里，而锁存器会“记住”刚才的电平，数码管就会一直亮着。
    }
}

Saved main.c successfully (overwritten).


### 一位数码管静态显示（封装函数）

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};//低电平点亮


void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void SEG_Show_1Digit(u8 pos, u8 num);
void Delay_ms(u16 xms); //@11.0592MHz


void main()
{
    LED_Bit_Set(0x00);

    while (1)
    {
        SEG_Show_1Digit(0, 1);
        Delay_ms(1);
    } 
}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}


void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void SEG_Show_1Digit(u8 pos, u8 num)
{
    if(num > 9) num = 0;

    BusWrite(LATCH_SEG, 0xFF);                 // 消隐
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));    // 选位
    BusWrite(LATCH_SEG, SEG_TAB[num]);         // 出段码
}

### 多位数码管静态显示（伪静态）
- 延时函数实现

%%c main
#include <STC15F2K60S2.H>
#include <intrins.h>

#define LATCH_DIG 0xC0  // 位选门
#define LATCH_SEG 0xE0  // 段选门

typedef unsigned char u8;

// 数字 1-8 的段码表
u8 code SEG_TAB[] = {0xf9, 0xa4, 0xb0, 0x99, 0x92, 0x82, 0xf8, 0x80};
// 1-8 位的位选码
u8 code DIG_TAB[] = {0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80};

/**
 * 核心写总线函数
 */
void BusWrite(u8 addr, u8 dat)
{
    P0 = dat;
    P2 = addr;
    _nop_();
    P2 = 0x00;
}

/**
 * 简单的延时函数
 * 用于控制每一位亮的时长。如果太长会闪烁，太短会看不清。
 */
void Delay1ms()
{
    unsigned char i, j;
    i = 11; j = 190;
    do { while (--j); } while (--i);
}

void main(void)
{
    u8 i;

    while(1)
    {
        for(i = 0; i < 8; i++)
        {
            // 1. 消隐：先把灯全关了，防止切换时产生“鬼影”
            BusWrite(LATCH_SEG, 0xFF); 

            // 2. 选位：选中第 i 个数码管
            BusWrite(LATCH_DIG, DIG_TAB[i]); 

            // 3. 出数：显示对应的数字
            BusWrite(LATCH_SEG, SEG_TAB[i]); 

            // 4. 停留：让这一位亮 1 毫秒
            Delay1ms(); 
        }
    }
}

### 多位静态数码管显示，封装成函数

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};//低电平点亮


void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void SEG_Show_Digit(u8 pos, u8 num);
void Delay_ms(u16 xms); //@11.0592MHz
void SEG_Show_Digit_2_Static(u8 pos, u8 num);

void main()
{
    u8 num = 0;
    LED_Bit_Set(0x00);

    while (1)
    {
        SEG_Show_Digit_2_Static(0, 92);
    }
}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void SEG_Show_Digit(u8 pos, u8 num)
{
    if(num > 9) num = 0;

    BusWrite(LATCH_SEG, 0xFF);                 // 消隐
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));    // 选位
    BusWrite(LATCH_SEG, SEG_TAB[num]);         // 出段码
}

void SEG_Show_Digit_2_Static(u8 pos, u8 num)
{
    u8 tens = num / 10 % 10;
    u8 ones = num % 10;
    SEG_Show_Digit(pos, tens);
    Delay_ms(1);
    SEG_Show_Digit(pos + 1, ones);
    Delay_ms(1);
}
//dynamic 动态
//static  静态

## 动态显示讲解

### 基础 一位一位地跳

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};

void BusWrite(u8 addr, u8 dat);
void SEG_Static_Show(u8 pos, u8 sep_num); // 序号
void Delay_ms(u16 xms);

void main()
{
    u8 pos = 0, sep = 0;
    BusWrite(LATCH_LED, ~0x00);
    while(1)
    {
        SEG_Static_Show(pos, sep);
        pos++, sep++;
        if (pos > 7)
            pos = 0;
        if (sep > 7)
            sep = 0;
        Delay_ms(900);
    }
}

void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void SEG_Static_Show(u8 pos, u8 sep_num)//序号
{
    if(sep_num > 9)
        sep_num = 0;
    BusWrite(LATCH_SEG, 0xFF);
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));
    BusWrite(LATCH_SEG, SEG_TAB[sep_num]);
}

### 进阶 多位数码管+动态+未封装

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};

void BusWrite(u8 addr, u8 dat);
void SEG_Static_Show(u8 pos, u8 sep_num); // 序号
void Delay_ms(u16 xms);

void main()
{
    u16 num = 0, tens = 0, units = 0;
    u16 t = 0, duration = 200;
    BusWrite(LATCH_LED, ~0x00);
    while(1)
    {
        for (num = 0; num < 99; num++)
        {
            tens = num / 10 % 10;
            units = num % 10;
            for (t = 0; t < duration; t ++)
            {
                SEG_Static_Show(0, tens);
                Delay_ms(2);
                SEG_Static_Show(1, units);
                Delay_ms(2);
            }
        }
        
    }
}

void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void SEG_Static_Show(u8 pos, u8 sep_num)//序号
{
    if(sep_num > 9)
        sep_num = 0;
    BusWrite(LATCH_SEG, 0xFF);
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));
    BusWrite(LATCH_SEG, SEG_TAB[sep_num]);
}

### 动态显示两位顺计时

#### 未封装

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};//低电平点亮


void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void SEG_Show_Digit(u8 pos, u8 num);
void Delay_ms(u16 xms); //@11.0592MHz
void SEG_Show_Digit_2_Static(u8 pos, u8 num);

void main()
{
    u8 num = 0;
    u16 time_tick = 0;
    LED_Bit_Set(0x00);

    while (1)
    {
        SEG_Show_Digit_2_Static(0, num);
        time_tick += 2;
        if(time_tick > 200)
        {
            time_tick = 0;
            num++;
            if (num > 99)
                num = 0;
        }
    }
}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void SEG_Show_Digit(u8 pos, u8 num)
{
    if(num > 9) num = 0;

    BusWrite(LATCH_DIG, 0x00);                 // 关位
    BusWrite(LATCH_SEG, SEG_TAB[num]);         // 出段码
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));    // 选位
    
}

void SEG_Show_Digit_2_Static(u8 pos, u8 num)
{
    u8 tens = num / 10 % 10;
    u8 ones = num % 10;
    SEG_Show_Digit(pos, tens);
    Delay_ms(1);
    SEG_Show_Digit(pos + 1, ones);
    Delay_ms(1);
}
//dynamic 动态
//static  静态

#### 封装1：
封装整个顺计时函数,并用计时变量计时

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};//低电平点亮


void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void SEG_Show_Digit(u8 pos, u8 num);
void Delay_ms(u16 xms); //@11.0592MHz
void SEG_Show_Digit_2_Static(u8 pos, u8 num);
void SEG_Show_Digit_2_Sequential_timing(u8 pos, u16 T_period_ms);


void main()
{
    u8 num = 0;
    u16 time_tick = 0;
    LED_Bit_Set(0x00);

    while (1)
    {
        SEG_Show_Digit_2_Sequential_timing(2, 200);
    }
}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void SEG_Show_Digit(u8 pos, u8 num)
{
    if(num > 9) num = 0;

    BusWrite(LATCH_DIG, 0x00);                 // 关位
    BusWrite(LATCH_SEG, SEG_TAB[num]);         // 出段码
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));    // 选位
    
}

void SEG_Show_Digit_2_Static(u8 pos, u8 num)
{
    u8 tens = num / 10 % 10;
    u8 ones = num % 10;
    SEG_Show_Digit(pos, tens);
    Delay_ms(1);
    SEG_Show_Digit(pos + 1, ones);
    Delay_ms(1);
}
//dynamic 动态
//static  静态

void SEG_Show_Digit_2_Sequential_timing(u8 pos, u16 T_period_ms)
{
    static u16 tick = 0, num = 0;
    SEG_Show_Digit_2_Static(pos, num);
    tick += 2;
    if (tick >= T_period_ms)
    {
        tick = 0;
        num++;
        if(num > 99)
            num = 0;
    }
}
//循环周期period



#### 封装2：
显示两位数字一段时间后消失，封转函数版本

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

static const u8 SEG_TAB[10] = {
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, 0x80, 0x90
};//低电平点亮


void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void SEG_Show_Digit(u8 pos, u8 num);
void Delay_ms(u16 xms); //@11.0592MHz
void SEG_Show_Digit_2_Static(u8 pos, u8 num);
void SEG_Show_Digit_2_Sequential_timing(u8 pos, u16 T_period_ms);
void SEG_Show_Digit_2_Static_DurationTime(u8 pos, u8 num, u16 duration_ms);

void main()
{
    u8 num = 0;
    u16 time_tick = 0;
    LED_Bit_Set(0x00);

    while (1)
    {
        SEG_Show_Digit_2_Static_DurationTime(1, 23, 1000);
    }
}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }
}

void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void SEG_Show_Digit(u8 pos, u8 num)
{
    if(num > 9) num = 0;

    BusWrite(LATCH_DIG, 0x00);                 // 关位
    BusWrite(LATCH_SEG, SEG_TAB[num]);         // 出段码
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));    // 选位
    
}

void SEG_Show_Digit_2_Static(u8 pos, u8 num)
{
    u8 tens = num / 10 % 10;
    u8 ones = num % 10;
    SEG_Show_Digit(pos, tens);
    Delay_ms(1);
    SEG_Show_Digit(pos + 1, ones);
    Delay_ms(1);
}
//dynamic 动态
//static  静态

void SEG_Show_Digit_2_Static_DurationTime(u8 pos, u8 num, u16 duration_ms)
{
    static u16 tick = 0;
    static u8 done = 0;
    if (done == 0)
    {
        SEG_Show_Digit_2_Static(pos, num);
        tick += 2;
        if(tick >= duration_ms)
        {
            tick = 0;
            done = 1;
        }
    }
    else
    {
        BusWrite(LATCH_DIG, 0x00);
        BusWrite(LATCH_SEG, 0xFF);
    }
}

void SEG_Show_Digit_2_Sequential_timing(u8 pos, u16 T_period_ms)
{
    static u16 Time_Tick = 0, num = 0;
    SEG_Show_Digit_2_Static(pos, num);
    Time_Tick += 2;
    if (Time_Tick >= T_period_ms)
    {
        Time_Tick = 0;
        num++;
        if(num > 99)
            num = 0;
    }
}
//循环周期period



#### 封装3:（按键控制正计时）
- 小实现
- 按键控制正计时
- 1开始 2暂停 3息屏

In [ ]:
%%c main
#include <STC15F2K60S2.H>
#include <INTRINS.H>

#define LATCH_LED 0x80
#define LATCH_DIG 0xC0
#define LATCH_SEG 0xE0

typedef unsigned char u8;
typedef unsigned int u16;
typedef unsigned long u32;

typedef enum
{
    Nixie_State_ScreenOff = 0,
    Nixie_State_Run,
    Nixie_State_Pause
} Nixie_State;

typedef struct
{
    u16 Key_Down;
    u16 Key_Up;
    u16 Key_Val;
} Key_State;

static const u8 SEG_TAB[10] = {
    0xC0,0xF9,0xA4,0xB0,0x99,0x92,0x82,0xF8,0x80,0x90
};




void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void LED_Index_Set(u8 index, bit on);
u16 Key_ScanMap(void);
u8 Key_ScanRead(void);
Key_State Key_Update(void);
bit Key_IsDown(Key_State *k, u16 index);
bit Key_IsUp(Key_State *k, u16 index);
void Delay_ms(u16 xms);
void SEG_StaticShow_1Digit(u8 pos, u8 num);
void SEG_StaticShow_2Digit(u8 pos, u8 num);
void SEG_DurationShow_1Digit(u8 pos, u8 num, u8 duration_ms);
void SEG_DurationShow_2Digit(u8 pos, u8 num, u8 duration_ms);

void main()
{
    Nixie_State NS = Nixie_State_ScreenOff;
    u16 key_num = 0;
    u16 num_show = 0;
    u16 tick_ms = 0;
    bit running = 0;
    Key_State k;
    LED_Bit_Set(0x00);
    BusWrite(LATCH_DIG, 0x00);
    while(1)
    {
        k = Key_Update();
        if(Key_IsDown(&k, 1))
        {
            num_show = 0;
            tick_ms = 0;
            NS = Nixie_State_Run;
        }
        if(Key_IsDown(&k, 2))
        {
            if(NS == Nixie_State_Run)
                NS = Nixie_State_Pause;
            else if(NS == Nixie_State_Pause)
                NS = Nixie_State_Run;
        }
        if(Key_IsDown(&k, 3))
        {
            NS = Nixie_State_ScreenOff;
        }

        switch (NS)
        {
        case Nixie_State_ScreenOff:
            BusWrite(LATCH_DIG, 0x00);
            Delay_ms(1);
            break;
        case Nixie_State_Run:
            SEG_StaticShow_2Digit(0, num_show);
            tick_ms += 2;
            if (tick_ms > 300)
            {
                tick_ms = 0;
                num_show++;
                if(num_show > 99)
                    num_show = 0;
            }
        
        case Nixie_State_Pause:
            SEG_StaticShow_2Digit(0, num_show);
            break;
        default:
            break;
        }

    }
}

void Delay_ms(u16 xms)		//@11.0592MHz
{
    while(xms --)
    {
        unsigned char i, j;
        _nop_();
        _nop_();
        _nop_();
        i = 11;
        j = 190;
        do
        {
            while (--j);
        } while (--i);
    }

}


void BusWrite(u8 addr, u8 dat)
{
    P2 = addr;
    P0 = dat;
    _nop_();
    P2 = 0x00;
}

void LED_Bit_Set(u8 dat)
{
    BusWrite(LATCH_LED, ~dat);
}

void LED_Index_Set(u8 index, bit on)
{
    static u8 LED_buf = 0;
    if (index >= 8)
        return;
    if (on)
        LED_buf |= (0x01 << index);
    else
        LED_buf &= ~(0x01 << index);
    LED_Bit_Set(LED_buf);
}

u8 Key_ScanRead(void)
{
    u8 key = 0;
    u8 col = 0, row = 0;
    P30 = 1, P31 = 1, P32 = 1, P33 = 1;
    for (col = 0; col < 4; col ++)
    {
        P44 = 1, P42 = 1, P35 = 1, P34 = 1;
        if(col == 0)
           P44 = 0;
        else if(col == 1)
           P42 = 0;
        else if(col == 2)
           P35 = 0;
        else if(col == 3)
           P34 = 0;

        if(!P30)
            row = 0;
        else if(!P31)
            row = 1;
        else if(!P32)
            row = 2;
        else if(!P33)
            row = 3;    
        if (P30 == 0 || P31 == 0 || P32 == 0 || P33 == 0)
            return key = row * 4 + col + 1;
    }
    return 0;
}

u16 Key_ScanMap(void)
{
    u16 map = 0;
    u8 col = 0, row = 0;
    P30 = 1, P31 = 1, P32 = 1, P33 = 1;
    for (col = 0; col < 4; col ++)
    {
        P44 = 1, P42 = 1, P35 = 1, P34 = 1;
        if(col == 0)
           P44 = 0;
        else if(col == 1)
           P42 = 0;
        else if(col == 2)
           P35 = 0;
        else if(col == 3)
           P34 = 0;

        if(!P30)
            map |= (1u << 0 * 4 + col);
        if(!P31)
            map |= (1u << 1 * 4 + col);
        if(!P32)
            map |= (1u << 2 * 4 + col);
        if(!P33)
            map |= (1u << 3 * 4 + col);
    }
    return map;
}

Key_State Key_Update(void)
{
    static u16 Old = 0;
    Key_State k;
    k.Key_Val = Key_ScanMap();
    k.Key_Down = k.Key_Val & (k.Key_Val ^ Old);
    k.Key_Up = (~k.Key_Val) & (k.Key_Val ^ Old);
    Old = k.Key_Val;
    return k;
}

bit Key_IsDown(Key_State *k, u16 index)
{
    if(index == 0 || index > 16)
        return 0;
    return ((k->Key_Down & (1u << (index - 1))) != 0);
}

bit Key_IsUp(Key_State *k, u16 index)
{
    if(index == 0 || index > 16)
        return 0;
    return ((k->Key_Up & (1u << (index - 1))) != 0);
}

void SEG_StaticShow_1Digit(u8 pos, u8 num)
{
    if(num > 9)
        num = 0;
    BusWrite(LATCH_DIG, 0x00);
    BusWrite(LATCH_SEG, SEG_TAB[num]);
    BusWrite(LATCH_DIG, (u8)(0x01 << pos));
    Delay_ms(1);
}

void SEG_StaticShow_2Digit(u8 pos, u8 num)
{
    u8 tens;
    u8 ones;
    if(num > 99)
        num = 0;
    tens = num / 10 % 10;
    ones = num % 10;
    SEG_StaticShow_1Digit(pos, tens);
    SEG_StaticShow_1Digit(pos + 1, ones);
}

void SEG_DurationShow_1Digit(u8 pos, u8 num, u8 duration_ms)
{
    u16 t = 0;
    if(num > 9)
        num = 0;
    for (t = 0; t < duration_ms; t ++)
    {
        SEG_StaticShow_1Digit(pos, num);
    }
}

void SEG_DurationShow_2Digit(u8 pos, u8 num, u8 duration_ms)
{
    u16 t = 0;
    if(num > 99)
        num = 0;
    for (t = 0; t < duration_ms; t ++)
    {
        SEG_StaticShow_2Digit(pos, num);
    }
}


### 进阶 多数码管+动态+定时器

In [ ]:
%%c main
#include <STC15F2K60S2.H> // 包含 STC15 系列单片机寄存器定义
#include <intrins.h>      // 提供 _nop_() 指令，用于极短的硬件缓冲延时

/* =========================================================================
 * 1. 硬件地址定义
 * =========================================================================
 */
#define LATCH_LED   0x80  // LED 锁存器地址 (Y4选通)
#define LATCH_DIG   0xC0  // 数码管位置锁存器 (Y6选通)
#define LATCH_SEG   0xE0  // 数码管段码锁存器 (Y7选通)

typedef unsigned char  u8;  // 8位无符号字符 (0~255)
typedef unsigned int   u16; // 16位无符号整数 (0~65535)

/* =========================================================================
 * 2. 全局变量与显存（系统的影子）
 * =========================================================================
 */
#define DIGITS 8 

// 【数码管显存】：这是显示的核心。中断函数每 0.5ms 会来这里“取货”并显示。
static volatile u8  G_Seg_Buf[DIGITS]; 

// 【扫描指针】：指向当前正在显示哪一个数码管位 (0~7)。
static volatile u8  G_Scan_Pos = 0;

// 【软件定时器】：用于在 0.5ms 的中断里数数，数满 200 次即为 100ms。
static volatile u16 G_Tick_Cnt = 0;    
static volatile bit G_100ms_Flag = 0; // 标志位：1 表示 100ms 时间到。

// 【LED 状态记录】：记住当前 LED 灯的亮灭情况。
static u8 G_LED_State = 0x00;

/* =========================================================================
 * 3. 翻译手册（码表）
 * =========================================================================
 */
// 数字 0-F 的段码表（低电平点亮，0亮1灭）
static const u8 SEG_TAB[16] =
{
    0xC0, 0xF9, 0xA4, 0xB0, 0x99, 0x92, 0x82, 0xF8, // 0-7
    0x80, 0x90, 0x88, 0x83, 0xC6, 0xA1, 0x86, 0x8E  // 8-F
};

// 数码管位置码表（1~8位）
static const u8 DIG_TAB[DIGITS] =
{
    0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80
};

/* =========================================================================
 * 4. 函数原型声明（解决 C206 报错，告诉编译器这些函数在后面）
 * =========================================================================
 */
void BusWrite(u8 addr, u8 dat);
void LED_Bit_Set(u8 dat);
void LED_Index_Set(u8 index, u8 on);
void SEG_Clear(void);
void SEG_Digit_Edit(u8 pos, u8 val, bit dot);
void SEG_Digit_Set(u8 pos);
void Timer0_Init_0p5ms_11M0592_12T(void);

/* =========================================================================
 * 5. 主程序入口 (main)
 * =========================================================================
 */
void main(void)
{
    u8 i;
    u8 decimal_count = 0; // 专门用于数码管显示的十进制计数器

    // --- 第一步：上电状态清理 ---
    LED_Bit_Set(0x00);   // 关闭所有 LED
    SEG_Clear();         // 清空所有数码管（全灭）

    // --- 第二步：准备初始背景内容 ---
    // 先让前 6 位数码管显示 1 2 3 4 5 6 固定不变
    for(i = 0; i < 6; i++)
        SEG_Digit_Edit(i, i + 1, 0);

    // --- 第三步：启动定时器（配置并点火） ---
    Timer0_Init_0p5ms_11M0592_12T(); 

    // --- 第四步：进入工作循环 ---
    while(1)
    {
        if(G_100ms_Flag) // 每 100 毫秒执行一次任务
        {
            static u8 led_step = 0; // 记录 LED 跑到了哪一步
            G_100ms_Flag = 0;       // 清除标志，等待下个 100ms

            /* 【任务 A：单灯流动效果】 */
            // 关键点：每次进来先全灭 LED，防止之前的灯重叠（解决 FF 问题）
            LED_Bit_Set(0x00); 
            // 点亮下一个灯 (1-8 循环)
            LED_Index_Set((led_step % 8) + 1, 1); 
            led_step++;

            /* 【任务 B：十进制计数器】 */
            decimal_count++;
            if(decimal_count > 99) decimal_count = 0; // 到 100 就归零

            // 将 decimal_count 拆成十位和个位，显示在最后两个位置
            SEG_Digit_Edit(6, decimal_count / 10, 0); // 十位数字
            SEG_Digit_Edit(7, decimal_count % 10, 0); // 个位数字
        }
    }
}

/* =========================================================================
 * 6. 底层函数具体实现
 * =========================================================================
 */

/**
 * @brief 总线写入函数：控制锁存器。
 */
void BusWrite(u8 addr, u8 dat)
{
    P0 = dat;        // 先摆放数据
    P2 = addr;       // 打开闸门
    _nop_();         // 极短等待，让电平稳定
    P2 = 0x00;       // 关闭闸门（数据已被锁住）
}

/**
 * @brief 更新 LED 的全局状态（影子变量）。
 */
void LED_Bit_Set(u8 dat)
{
    G_LED_State = dat; // 同步影子变量
    BusWrite(LATCH_LED, (u8)~G_LED_State); // ~ 取反是因为硬件低电平亮
}

/**
 * @brief 精确控制某一个 LED 的亮灭。
 * @param index: 1~8 对应 L1~L8。
 */
void LED_Index_Set(u8 index, u8 on)
{
    if(index < 1 || index > 8) return;

    if(on)  G_LED_State |=  (1u << (index - 1)); // 将对应位置 1
    else    G_LED_State &= ~(1u << (index - 1)); // 将对应位置 0

    BusWrite(LATCH_LED, (u8)~G_LED_State); 
}

/**
 * @brief 清空显存（数码管全灭）。
 */
void SEG_Clear(void)
{
    u8 i;
    for(i = 0; i < DIGITS; i++)
        G_Seg_Buf[i] = 0xFF; 
}

/**
 * @brief 修改显存中的一个数字。
 * @param pos: 哪个管子(0~7)。 @param val: 哪个数(0~15)。 @param dot: 亮不亮小数点。
 */
void SEG_Digit_Edit(u8 pos, u8 val, bit dot)
{
    u8 seg;
    if(pos >= DIGITS) return; 

    seg = SEG_TAB[val & 0x0F]; // 查表找形状

    if(dot)  seg &= 0x7F; // bit7 强制为 0 (点亮 dp)
    else     seg |= 0x80; // bit7 强制为 1 (熄灭 dp)

    G_Seg_Buf[pos] = seg; // 存入数组
}

/**
 * @brief 物理刷新函数：解决鬼影的关键三步。
 */
void SEG_Digit_Set(u8 pos)
{
    BusWrite(LATCH_SEG, 0xFF);         // 1. 消隐：全灭
    BusWrite(LATCH_DIG, DIG_TAB[pos]); // 2. 选位：选中下一个
    BusWrite(LATCH_SEG, G_Seg_Buf[pos]);// 3. 出数：点亮
}

/**
 * @brief 定时器 0 初始化：封装版。
 */
void Timer0_Init_0p5ms_11M0592_12T(void)
{
    AUXR &= ~0x80; // T0 为 12T 模式
    TMOD &= 0xF0;  // 模式 1 (16位)
    TMOD |= 0x01;		
    TH0 = 0xFE;    // 0.5ms 初值
    TL0 = 0x33;		
    ET0 = 1;       // 允许 T0 中断
    EA  = 1;       // 开启总电闸
    TR0 = 1;       // 启动计时
}

/* =========================================================================
 * 7. 中断服务程序（心脏跳动）
 * =========================================================================
 */
void Timer0_ISR(void) interrupt 1
{
    // A. 必须重装载，否则第二次中断间隔就不对了
    TH0 = 0xFE;
    TL0 = 0x33;

    // B. 扫描一位数码管（每 0.5ms 扫一位，4ms 扫完全部 8 位）
    SEG_Digit_Set(G_Scan_Pos);
    if(++G_Scan_Pos >= DIGITS)
    {
        G_Scan_Pos = 0;
        // 扫完一圈顺便重写一次 LED，防止 LED 亮度受到总线干扰而抖动
        BusWrite(LATCH_LED, (u8)~G_LED_State);
    }

    // C. 产生 100ms 的标志位信号
    if(++G_Tick_Cnt >= 200) // 0.5ms * 200 = 100ms
    {
        G_Tick_Cnt = 0;
        G_100ms_Flag = 1; 
    }
}

Saved main.c successfully (overwritten).


## 示例

In [18]:
%%c main
#include <STC15F2K60S2.H>
#include <intrins.h>

/* ===================== 锁存地址（常见 138+573 板子） ===================== */
#define LATCH_LED   0x80
#define LATCH_DIG   0xC0
#define LATCH_SEG   0xE0

typedef unsigned char  u8;
typedef unsigned int   u16;

/* ===================== 数码管配置 ===================== */
#define DIGITS 8

/* ===================== 全局变量（影子寄存器） ===================== */
static volatile u8  G_Seg_Buf[DIGITS];
static volatile u8  G_Scan_Pos = 0;

static volatile u16 G_Tick_Cnt = 0;     // 0.5ms 计数
static volatile bit G_100ms_Flag = 0;

static u8 G_LED_State = 0x00;

/* ===================== 段码 / 位码表（共阴，dp 为 bit7，0亮1灭） ===================== */
static const u8 SEG_TAB[16] =
{
    0xC0,0xF9,0xA4,0xB0,
    0x99,0x92,0x82,0xF8,
    0x80,0x90,0x88,0x83,
    0xC6,0xA1,0x86,0x8E
};

static const u8 DIG_TAB[DIGITS] =
{
    0x01,0x02,0x04,0x08,
    0x10,0x20,0x40,0x80
};

/* ===================== 函数声明 ===================== */
void BusWrite(u8 addr, u8 dat);

void LED_Bit_Set(u8 dat);
void LED_Index_Set(u8 index, u8 on);

void SEG_Clear(void);
void SEG_Digit_Edit(u8 pos, u8 val, bit dot);
void SEG_Digit_Set(u8 pos);

void Timer0_Init_0p5ms_11M0592_12T(void);

/* ===================== main ===================== */
void main(void)
{
    u8 i;

    LED_Bit_Set(0x00);
    SEG_Clear();

    /* 上电测试：12345678 */
    for(i = 0; i < DIGITS; i++)
        SEG_Digit_Edit(i, i + 1, 0);

    Timer0_Init_0p5ms_11M0592_12T();

    while(1)
    {
        if(G_100ms_Flag)
        {
            static u8 step = 0;
            G_100ms_Flag = 0;

            /* LED 跑马灯 */
            LED_Bit_Set(0x00);
            LED_Index_Set((step % 8) + 1, 1);
            step++;

            /* 数码管右两位显示 LED 状态（高4位/低4位） */
            SEG_Digit_Edit(6, (G_LED_State >> 4) & 0x0F, 0);
            SEG_Digit_Edit(7,  G_LED_State & 0x0F, 0);
        }
    }
}

/* ===================== 总线写（关键稳定版：先数据后锁存） ===================== */
void BusWrite(u8 addr, u8 dat)
{
    P0 = dat;        // 先放数据
    P2 = addr;       // 再选锁存
    P2 = 0x00;
}

/* ===================== LED ===================== */
void LED_Bit_Set(u8 dat)
{
    G_LED_State = dat;
    BusWrite(LATCH_LED, (u8)~G_LED_State);  // 常见板子 LED 为低亮
}

void LED_Index_Set(u8 index, u8 on)
{
    if(index < 1 || index > 8) return;

    if(on)  G_LED_State |=  (1u << (index - 1));
    else    G_LED_State &= ~(1u << (index - 1));

    BusWrite(LATCH_LED, (u8)~G_LED_State);
}

/* ===================== 数码管 ===================== */
void SEG_Clear(void)
{
    u8 i;
    for(i = 0; i < DIGITS; i++)
        G_Seg_Buf[i] = 0xFF;     // 全灭
}

void SEG_Digit_Edit(u8 pos, u8 val, bit dot)
{
    u8 seg;
    if(pos >= DIGITS) return; // 安全检查：位置不能超过 8

    // 1. 查表：把数字(如 5) 变成段码(如 0x92)
    // val & 0x0F 是为了防止传入的数字超过 15 (F)
    seg = SEG_TAB[val & 0x0F]; 

    /* 2. 处理小数点 (dp 是第 7 位) */
    /* 我们用的数码管是 0 亮 1 灭 */
    if(dot) 
        seg &= 0x7F; // 0x7F 是 0111 1111。用“与”运算强制把第 7 位(dp)变成 0，点亮它。
    else    
        seg |= 0x80; // 0x80 是 1000 0000。用“或”运算强制把第 7 位(dp)变成 1，熄灭它。

    // 3. 填入显存
    G_Seg_Buf[pos] = seg; 
}

void SEG_Digit_Set(u8 pos)
{
    /* 三步法：段灭 → 选位 → 上段（减少鬼影） */
    BusWrite(LATCH_SEG, 0xFF);
    BusWrite(LATCH_DIG, DIG_TAB[pos]);
    BusWrite(LATCH_SEG, G_Seg_Buf[pos]);
}

/* ===================== Timer0 ===================== */
/*
 * 外部晶振 11.0592MHz，12T 模式：
 * 定时 0.5ms 需要计数约 460.8 -> 取 461
 * 重装值 = 65536 - 461 = 65075 = 0xFE33
 */
void Timer0_Init_0p5ms_11M0592_12T(void)
{
    AUXR &= ~0x80;      // Timer0 12T
    TMOD &= 0xF0;
    TMOD |= 0x01;       // 16位定时器

    TH0 = 0xFE;
    TL0 = 0x33;

    ET0 = 1;
    EA  = 1;
    TR0 = 1;
}

/* ===================== Timer0 ISR ===================== */
void Timer0_ISR(void) interrupt 1
{
    /* 重装 0.5ms */
    TH0 = 0xFE;
    TL0 = 0x33;

    /* 扫描当前位 */
    SEG_Digit_Set(G_Scan_Pos);

    if(++G_Scan_Pos >= DIGITS)
    {
        G_Scan_Pos = 0;

        /* 每帧兜底一次 LED（减少总线翻转导致的毛刺/闪烁） */
        BusWrite(LATCH_LED, (u8)~G_LED_State);
    }

    /* 100ms 软件定时：0.5ms * 200 ≈ 100ms */
    if(++G_Tick_Cnt >= 200)
    {
        G_Tick_Cnt = 0;
        G_100ms_Flag = 1;
    }
}


Saved main.c successfully (overwritten).
